In [2]:
# install pycomm3 and ipywidgets packages
# !pip install -q pycomm3 ipywidgets

In [1]:
import pycomm3
from pycomm3 import logger

logger = logger.configure_default_logger(level="VERBOSE", filename="pycomm3.log")

In [4]:
devices_list = pycomm3.CIPDriver.discover(broadcast_address="255.255.255.0")

2024-06-19 12:03:57,261 [INFO] pycomm3.cip_driver.CIPDriver.discover(): Discovering devices...
2024-06-19 12:03:57,261 [DEBUG] pycomm3.cip_driver.CIPDriver.discover(): Broadcasting discover for IP: 192.168.1.100
2024-06-19 12:03:57,262 [ERROR] pycomm3.cip_driver.CIPDriver._broadcast_discover(): Error broadcasting discover request
Traceback (most recent call last):
  File "c:\Program Files\Python311\Lib\site-packages\pycomm3\cip_driver.py", line 247, in _broadcast_discover
    sock.sendto(message, (broadcast_address, 44818))
OSError: [WinError 10051] A socket operation was attempted to an unreachable network
2024-06-19 12:03:57,263 [DEBUG] pycomm3.cip_driver.CIPDriver.discover(): No devices found so far, attempting broadcast without binding to an IP.
2024-06-19 12:03:57,264 [ERROR] pycomm3.cip_driver.CIPDriver._broadcast_discover(): Error broadcasting discover request
Traceback (most recent call last):
  File "c:\Program Files\Python311\Lib\site-packages\pycomm3\cip_driver.py", line 247

In [3]:
import socket

In [4]:
socket.getaddrinfo(socket.gethostname(), None)

[(<AddressFamily.AF_INET6: 23>,
  0,
  0,
  '',
  ('fe80::7ea9:5cc8:d888:66a4', 0, 0, 18)),
 (<AddressFamily.AF_INET: 2>, 0, 0, '', ('192.168.1.100', 0)),
 (<AddressFamily.AF_INET6: 23>,
  0,
  0,
  '',
  ('fd12:5e1d:bd43:fe40:48c1:b30:464d:1002', 0, 0, 0)),
 (<AddressFamily.AF_INET6: 23>,
  0,
  0,
  '',
  ('fd12:5e1d:bd43:fe40:2dea:5be7:791c:ed0c', 0, 0, 0))]

In [ ]:
socket.socket(socket.AF_INET, socket.SOCK_STREAM)

In [2]:


# Discover the devices on the network
devices_list = CIPDriver.discover()

# if the devices_list is empty
if not devices_list:
    print('No devices found')
else:
    for device in devices_list:
        print(device)

No devices found


In [7]:
import socket
import logging

# Setup logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

class CIPClient:
    def __init__(self, ip, port, timeout=5):
        self.ip = ip
        self.port = port
        self.timeout = timeout
        self.sock = None

    def connect(self):
        try:
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock.settimeout(self.timeout)
            logger.debug(f"Connecting to {self.ip}:{self.port}")
            self.sock.connect((self.ip, self.port))
            logger.debug("Connection established")
            return True
        except Exception as e:
            logger.error(f"Failed to connect to {self.ip}:{self.port} - {e}")
            return False

    def listen_for_reply(self):
        try:
            while True:
                data = self.sock.recv(1024)
                if not data:
                    break
                logger.debug(f"Received data: {data}")
                print(f"Received: {data}")
        except socket.timeout:
            logger.debug("Socket timeout, no more data.")
        except Exception as e:
            logger.error(f"Error receiving data - {e}")
        finally:
            self.close()

    def close(self):
        if self.sock:
            self.sock.close()
            logger.debug("Socket closed")


ip_address = "192.168.1.100"  # Replace with the actual IP address
port = 44818  # Replace with the actual port number from the EDS file
client = CIPClient(ip_address, port)
if client.connect():
    client.listen_for_reply()

DEBUG:__main__:Connecting to 192.168.1.100:44818
ERROR:__main__:Failed to connect to 192.168.1.100:44818 - [WinError 10061] No connection could be made because the target machine actively refused it


In [3]:
class PumpController:
    """
    A PumpController class to handle communication with a pump using EtherNet/IP 
    protocol. It allows reading and writing parameters to the pump.
    """
    def __init__(self, ip_address):
        """
        Initializes the PumpController with the given IP address of the pump.

        Parameters:
        ip_address (str): The IP address of the pump.
        """
        self.ip_address = ip_address

        #! potential error here, don't how the tag list is initialized automatically or not
        #! or maybe the pump don't work with this package at all, then we have to fall back to the CIPDriver
        with pycomm3.LogixDriver(ip_address, init_tags=True) as pump:
            self.pump = pump
        #! potential error here, don't how the tag list is initialized automatically or not

        # I don't if open() is necessary here, official doc doesn't mention it
        open_return = self.pump.open()
        print(f"Open return: {open_return}")

        # Check if the connection was successful
        if self.pump.connected:
            print(f"Connected to pump at {ip_address}")

            # Print PLC info
            plc_info = self.pump.get_plc_info()

            print("PLC Info:")
            for key, value in plc_info.items():
                print(f"{key}: {value}")
        else:
            print(f"Failed to connect to pump at {ip_address}")

# if the device is not empty, we will attempt to create a PumpController object for each device
for device in devices_list:
    ip_address = device['ip_address']
    pump_controller = PumpController(ip_address)